## 🧐 Klasyfikacja nowego obrazu

Teraz załadujemy pojedyncze zdjęcie i wprowadzimy do modelu, aby sprawdzić, jaki typ komórki zostanie przewidziany.  

Model zwróci **wektor prawdopodobieństw** dla 4 klas, a my wybierzemy tę z najwyższą wartością.


In [1]:
import tensorflow as ts
import numpy as np
from tensorflow.keras.preprocessing import image

# Wczytanie modelu
model = ts.keras.models.load_model("blood_cell_classifier.h5")

# Przetwarzanie obrazu do testu
img_path = "images/TEST/EOSINOPHIL/sample.jpg"
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predykcja
predictions = model.predict(img_array)
class_names = ["EOSINOPHIL", "LYMPHOCYTE", "MONOCYTE", "NEUTROPHIL"]
predicted_class = class_names[np.argmax(predictions)]
print(f"Model przewiduje: {predicted_class}")


2025-02-24 08:54:35.605840: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 08:54:35.639033: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-24 08:54:35.639058: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-24 08:54:35.639085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 08:54:35.645270: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 08:54:35.645685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 320, 240, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_4'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

## 🔎 Wykrywanie i liczenie komórek na zdjęciu

Po poprawnej klasyfikacji możemy wykrywać komórki na całym zdjęciu i je policzyć.  
Zastosujemy podejście klasyczne – **detekcję okręgów za pomocą OpenCV**:
1. Przekształcimy obraz do skali szarości.
2. Wygładzimy obraz filtrem Gaussa, aby usunąć szum.
3. Wykryjemy okręgi metodą **HoughCircles**, która identyfikuje koliste struktury (komórki).  
4. Naniesiemy wykryte okręgi na obraz i policzymy liczbę komórek.  


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Wczytanie obrazu
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Użycie filtru Gaussa i detekcja okręgów metodą HoughCircles
blurred = cv2.GaussianBlur(gray, (11,11), 0)
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                           param1=50, param2=30, minRadius=10, maxRadius=40)

# Rysowanie wykrytych komórek
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)

# Liczba wykrytych komórek
print(f"Wykryto {len(circles[0])} komórek krwi.")

# Wyświetlenie wyników
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()